# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [4]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [8]:
document = ET.parse( './data/mondial_database.xml' )
import pandas as pd

In [27]:
full_list=[]
for element in document.iterfind('country'):
    record=[]
    if element.find('infant_mortality') is not None:
        record.append(element.find('name').text)
        record.append(float(element.find('infant_mortality').text))
        full_list.append(record)
df=pd.DataFrame(full_list,columns=['country','infant_mortality'])

In [30]:
df=df.sort_values(['infant_mortality'])
df.head(10)

,country,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [69]:
full_list=[]
for a in document.iterfind('.//city'):
    record=[]
    if a.find('name') is not None:
        record.append(a.find('name').text)
        c=0
        for b in a.iterfind('./population'):
            c=int(b.text)
        record.append(c)
    full_list.append(record)

df=pd.DataFrame(full_list,columns=['city','population'])   
df=df.sort_values(['population'],ascending=False)
df.head(10)

,city,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [81]:
full_list=[]
for a in document.iterfind('country'):
    nm=a.find('name').text
    for p in a.iterfind('population'):
        pass
    pop=int(p.text)
    for b in a.iterfind('ethnicgroup'):
        record=[]
        record.append(nm)
        record.append(pop)
        record.append(float(b.get('percentage')))
        record.append(b.text)
        full_list.append(record)
dfgroup=pd.DataFrame(full_list,columns=['country','population','percentage','group'])
dfgroup.head()

,country,population,percentage,group
0,Albania,2800138,95.0,Albanian
1,Albania,2800138,3.0,Greek
2,Greece,10816286,93.0,Greek
3,Macedonia,2059794,64.2,Macedonian
4,Macedonia,2059794,25.2,Albanian


In [89]:
df2=dfgroup.groupby(['group']).sum()
df2=df2.sort_values(['grouppop'],ascending=False)
df2.head(10)

,population,percentage,grouppop
group,,,
Han Chinese,1360720000,91.50,1.245059e+09
Indo-Aryan,1210854977,72.00,8.718156e+08
European,1157295639,970.82,4.948722e+08
African,975352746,1868.55,3.183251e+08
Dravidian,1210854977,25.00,3.027137e+08
Mestizo,279743964,870.70,1.577344e+08
Bengali,149772364,98.00,1.467769e+08
Russian,322438406,224.10,1.318570e+08
Japanese,127298000,99.40,1.265342e+08


In [101]:
full_list=[]
for a in document.iterfind('river'):
    record=[]
    record.append(a.find('name').text)
    record.append(a.get('country'))
    if a.find('length') is not None:
        record.append(float(a.find('length').text))
    full_list.append(record)
df=pd.DataFrame(full_list, columns=['name','country','length'])
df=df.sort_values('length',ascending=False)
df.head()

,name,country,length
174,Amazonas,CO BR PE,6448.0
137,Jangtse,CN,6380.0
136,Hwangho,CN,4845.0
123,Lena,R,4400.0
205,Zaire,RCB ZRE,4374.0


In [125]:
full_list=[]
for a in document.iterfind('airport'):
    record=[]
    record.append(a.find('name').text)
    record.append(a.get('country'))
    if a.find('elevation') is not None:
        if a.find('elevation').text is not None:
            record.append(float(a.find('elevation').text))
    full_list.append(record)
df=pd.DataFrame(full_list, columns=['name','country','elevation'])
df=df.sort_values('elevation',ascending=False)
df.head()

,name,country,elevation
80,El Alto Intl,BOL,4063.0
219,Lhasa-Gonggar,CN,4005.0
241,Yushu Batang,CN,3963.0
813,Juliaca,PE,3827.0
815,Teniente Alejandro Velasco Astete Intl,PE,3311.0
